In [1]:
from collections.abc import MutableMapping
import itertools
from enum import Enum
from typing import Optional

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets


In [104]:
class SingalGeneratorType(Enum):
    WaveGenerator = 0
    RTL_SDR = 1

class NoiseType(Enum):
    NoNoise = 0
    AdditiveWhiteGaussianNoise = 1


class WindowingApproach(Enum):
    NoWindowing = 0
    Hamming = 1

def flatten_dict(dictionary, parent_key='', separator='_'):
    items = []
    for key, value in dictionary.items():
        normalized_parent_key = parent_key
        if isinstance(parent_key, Enum):
            normalized_parent_key=parent_key.name
        new_key = normalized_parent_key + separator + str(key) if parent_key else key
        if isinstance(value, MutableMapping):
            items.extend(flatten_dict(value, new_key, separator=separator).items())
        else:
            items.append((new_key, value))
    return dict(items)

def convert_to_db(value:float, reference_value:float, is_power_or_energy_value: bool = True) -> float:
    '''
    Reference: https://dspillustrations.com/pages/posts/misc/decibel-conversion-factor-10-or-factor-20.html
    '''
    if is_power_or_energy_value:
        return 10*np.log10(value/reference_value)
    else:
        return 20*np.log10(value/reference_value)

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

def wrap_plot(ys: list[list[float]],
         x:Optional[list[float]]=None,
         x_ax_label:str = '',
         y_ax_label:str = '',
         y_labels:Optional[list[str]]=None,
         set_y_log_scale:bool = False,
         set_x_log_scale:bool = False,
         marker=None,
         linestyle:str='-') -> None:
    if x is None:
        x = np.arange( len(ys[0]))
    assert((y_labels == None) or (len(ys) == len(y_labels)))
    
    color_cycle = itertools.cycle(mplcolors.TABLEAU_COLORS)
    
    for i, (y, color) in enumerate(zip(ys, color_cycle)):
        assert(len(x) == len(y))
        line_label = y_labels[i] if y_labels else None 
        
        plt.plot(x, y, color=color, marker=marker, linestyle=linestyle, label=line_label)
        
    # plt.plot(f[np.argmax(X_mag)], np.max(X_mag), '') # show max
    plt.grid()
    
    if x_ax_label:
        plt.xlabel(x_ax_label)
    if y_ax_label:
        plt.ylabel(y_ax_label)
    
    if set_y_log_scale:
        plt.yscale('symlog')
    if set_x_log_scale:
        plt.xscale('symlog')
    
    plt.show()

def generate_simple_wave(
        amplitude:float,
        frequency: float,
        phase: float,
        t:np.ndarray=np.arange(101)) -> np.ndarray:
    return amplitude * np.sin(frequency*2*np.pi*t+phase)

def generate_awgn():
    pass



In [105]:
# Style/layout tweaks added to all widgets.
widget_disp = {'style':{'description_width': 'initial'},'layout':Layout(width='90%')}

# Parameters for various signal generators.
generator_parameters = {
    SingalGeneratorType.WaveGenerator : {
        'num_samples': widgets.IntSlider(value=1025, min=101, max=2049, step=10, **widget_disp),
        'sample_rate': widgets.FloatLogSlider(value=1e6, base=10, min=-10, max=10, step=0.2, **widget_disp),
        'amplitude_in_phase': widgets.FloatSlider(value=1.0, min=-10.0, max=10.0, step=0.1, **widget_disp),
        'frequency': widgets.FloatLogSlider(value=1e-8, base=10, min=-10, max=10, step=0.1, **widget_disp),
        'phase': widgets.FloatSlider(value=0.0, min=-np.pi, max=np.pi, step=0.0001, **widget_disp),
        'amplitude_quadriture': widgets.FloatSlider(value=1.0, min=-10.0, max=10.0, step=0.1, **widget_disp),
    },
}

# Parameters for controlling simulated noise.
noise_parameters ={
    NoiseType.AdditiveWhiteGaussianNoise : {
        'mean': widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, **widget_disp),
        'std': widgets.FloatSlider(value=0, min=1e-4, max=1, step=1e-2, **widget_disp),
    }
}

# Parameters for controlling simulated noise.
# For more info see: https://pysdr.org/content/frequency_domain.html#windowing
windowing_parameters = {
    'windowing_selector' : widgets.Dropdown(value=WindowingApproach.Hamming, options=WindowingApproach,**widget_disp)
}

filtering_parameters = {
    'squelch_floor' : widgets.FloatSlider(value=300, min=0, max=1000, step=5, **widget_disp)
}

def select_sample_generator(selected_generator: SingalGeneratorType) -> None:
    for gen_name, gen_parameters in generator_parameters.items():
        for _, param_widget in gen_parameters.items():
            if selected_generator == gen_name:
                param_widget.layout.visibility = 'visible'
            else:
                param_widget.layout.visibility = 'hidden'
    pass

sample_generator_selection_widget=widgets.Dropdown(options=SingalGeneratorType)
interact(select_sample_generator, x=sample_generator_selection_widget);


def generate_and_display_sample(**kwargs):
    samples = None
    signal_gen_params = generator_parameters[sample_generator_selection_widget.value]
    
    s = None
    if sample_generator_selection_widget.value == SingalGeneratorType.WaveGenerator:
        num_samples = signal_gen_params['num_samples'].value
        sample_rate = signal_gen_params['sample_rate'].value
        # t = np.arange(
        #         start=-np.ceil(num_samples/2),
        #         stop=np.ceil(num_samples/2)
        #     ) * sample_rate
        t = np.linspace(
                start=-np.ceil(num_samples/2),
                stop=np.ceil(num_samples/2),
                num=num_samples) * sample_rate
        
        amplitude_I = signal_gen_params['amplitude_in_phase'].value
        amplitude_Q = signal_gen_params['amplitude_quadriture'].value

        frequency = signal_gen_params['frequency'].value
        phase = signal_gen_params['phase'].value


        # For more info see: https://pysdr.org/content/sampling.html#quadrature_sampling
        s_I_component = amplitude_I * np.cos(2*np.pi*frequency*t)
        s_Q_component = amplitude_Q * np.sin(2*np.pi*frequency*t) # 90 degrees out of phase of I-component
        
        s = s_I_component + s_Q_component
        
        wrap_plot([s,s_I_component,s_Q_component],t,
                  x_ax_label='Time (seconds)',
                  y_labels=['Combinded', 'I-component(Cosine)', 'Q-component(Sine)'])
        
         
    else:
        pass    


    
    # Generate some gaussian white noise to simulate real world conditions.
    awgn_params = noise_parameters[NoiseType.AdditiveWhiteGaussianNoise]
    awgn = np.random.normal(loc=awgn_params['mean'].value,scale=awgn_params['std'].value, size=len(s))
    s = s + awgn

    
    # Select a Windowing
    if(windowing_parameters['windowing_selector'].value == WindowingApproach.Hamming):
        s = s * np.hamming(len(s))
    
    
    # FFT 
    #
    # S = np.fft.fft(s)
    # S = np.fft.fftshift(S)
    # freqs = np.linspace(-np.ceil(sample_rate/2),np.ceil(sample_rate/2), num_samples)

    # FFT: only non-negative frequencies since our input signal is real)
    S = np.fft.rfft(s)
    print('len(S):',len(S))
    freqs = np.linspace(0,np.ceil(sample_rate/2), num_samples//2+1)
    fft_mag = np.abs(S)
    fft_phase = np.angle(S)

    squelch_floor = filtering_parameters['squelch_floor'].value


    
    filtered_mag = fft_mag[fft_mag > squelch_floor]
    filtered_phase = fft_phase[fft_mag > squelch_floor]
    
    
    wrap_plot([fft_mag, np.full_like(fft_mag,squelch_floor)], freqs, x_ax_label='Frequency (Hz)', y_ax_label='FFT Magnitude', set_y_log_scale=False,set_x_log_scale=True)
    wrap_plot([fft_phase], freqs, x_ax_label='Frequency (Hz)', y_ax_label='FFT Phase', set_y_log_scale=False,set_x_log_scale=True)


    
    # Plot IQ Constellations 
    (iq_x, iq_y) = pol2cart(filtered_mag, filtered_phase)
    plt.scatter(iq_x, iq_y)
    
    
    pass

flattened_params = flatten_dict(generator_parameters)
flattened_params.update(flatten_dict(noise_parameters))
flattened_params.update(flatten_dict(windowing_parameters))
flattened_params.update(flatten_dict(filtering_parameters))


interact(generate_and_display_sample,**flattened_params);


interactive(children=(Dropdown(description='selected_generator', options={'WaveGenerator': <SingalGeneratorTyp…

interactive(children=(IntSlider(value=1025, description='WaveGenerator_num_samples', layout=Layout(visibility=…

In [ ]:
import sys
sys.path.append('/opt/homebrew/lib')

import os

# print('DYLD_LIBRARY_PATH before:',os.environ['DYLD_LIBRARY_PATH'])
# os.environ['LD_LIBRARY_PATH'] = os.getcwd()  # or whatever path you want

os.environ['LD_LIBRARY_PATH'] = '/opt/homebrew/lib'
# os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib'


In [ ]:
# from rtlsdr import RtlSdr

# sdr = RtlSdr()
# sdr.sample_rate = 2.048e6 # Hz
# sdr.center_freq = 100e6   # Hz
# sdr.freq_correction = 60  # PPM
# print(sdr.valid_gains_db)
# sdr.gain = 49.6
# print(sdr.gain)

# x = sdr.read_samples(4096)
# sdr.close()

# plt.plot(x.real)
# plt.plot(x.imag)
# plt.legend(["I", "Q"])
# # plt.savefig("../_images/rtlsdr-gain.svg", bbox_inches='tight')
# plt.show()
